In [ ]:
!wget http://images.cocodataset.org/zips/test2017.zip

In [ ]:
!unzip /content/test2017.zip 

# Get high res and low res images from the dataset

In [2]:
from numpy import array
from skimage.transform import resize as imresize
from keras.layers import Lambda
import tensorflow as tf
from skimage import data, io, filters
import numpy as np
from numpy import array
from numpy.random import randint
import os
import sys
import matplotlib.pyplot as plt
from PIL import Image

import cv2

Using TensorFlow backend.


In [3]:
# Takes list of images and provide HR images in form of array
def hr_images(images):
    images_hr = array(images)
    return images_hr
    

In [4]:
# Takes list of images and provide LR images in form of numpy array
def lr_images(images_real, downscale):
    images=[]
    for img in range(len(images_real)):
        images.append(cv2.resize(images_real[img], (images_real[img].shape[0]//downscale, images_real[img].shape[1]//downscale), interpolation=cv2.INTER_CUBIC))
    images_lr=array(images)
    return images_lr
        

# Data Preprocessing

In [5]:
def normalize(input_data):
    input_data = (input_data.astype(np.float32)-127.5)/127.5
    return input_data

In [6]:
def denormalize(input_data):
    input_data = (input_data + 1) *127.5
    return input_data.astype(np.uint8)

# Load Data

### Load training data

In [7]:
images=os.listdir(os.path.join("/content/test2017"))
print(len(images))
images[:10]

40670


['000000388673.jpg',
 '000000508745.jpg',
 '000000064778.jpg',
 '000000393443.jpg',
 '000000453960.jpg',
 '000000203924.jpg',
 '000000361663.jpg',
 '000000198791.jpg',
 '000000527986.jpg',
 '000000285582.jpg']

In [8]:
train_img=images[:1010]

In [9]:
#Only one directory
"""
def load_path(path):
    directories = []
    if os.path.isdir(path):
        directories.append(path)
    for elem in os.listdir(path):
        if os.path.isdir(os.path.join(path,elem)):
            directories=directories+load_path(os.path.join(path,elem))
            directories.append(os.path.join(path,elem))
    return directories
"""

'\ndef load_path(path):\n    directories = []\n    if os.path.isdir(path):\n        directories.append(path)\n    for elem in os.listdir(path):\n        if os.path.isdir(os.path.join(path,elem)):\n            directories=directories+load_path(os.path.join(path,elem))\n            directories.append(os.path.join(path,elem))\n    return directories\n'

In [10]:

def load_data_from_dirs(dirs,ext):
    files=[]
    for f in train_img:
        if f.endswith(ext):
            img=io.imread(os.path.join(dirs,f))
            if len(img.shape) > 2:
                files.append(img)
    return files


In [11]:
def load_data_from_dirs(dirs,ext):
    files=[]
    for f in train_img:
        if f.endswith(ext):
            img=np.array(Image.open(os.path.join(dirs,f)).resize((192,192)))
            if len(img.shape) > 2:
                files.append(img)
    return files


In [12]:
fil = load_data_from_dirs('/content/test2017/','.jpg')

In [13]:
def load_training_data(fil, number_of_images = 1000, train_test_ratio = 0.8):

    number_of_train_images = int(number_of_images * train_test_ratio)
    
    files = fil
    
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()
        
    
    test_array = array(files)
    if len(test_array.shape) < 3:
        print("Images are of not same shape")
        print("Please provide same shape images")
        sys.exit()
    
    
    x_train = files[:number_of_train_images]
    x_test = files[number_of_train_images:number_of_images]
    
    x_train_hr = hr_images(x_train)
    x_train_hr = normalize(x_train_hr)
    
    x_train_lr = lr_images(x_train, 4)
    x_train_lr = normalize(x_train_lr)
    
    x_test_hr = hr_images(x_test)
    x_test_hr = normalize(x_test_hr)
    
    x_test_lr = lr_images(x_test, 4)
    x_test_lr = normalize(x_test_lr)
    
    return x_train_lr, x_train_hr, x_test_lr, x_test_hr

In [14]:
x_train_lr,x_train_hr,x_test_lr,x_test_hr=load_training_data(fil, number_of_images = 1000, train_test_ratio = 0.8)

In [15]:
print(x_train_lr.shape,x_train_hr.shape,x_test_lr.shape,x_test_hr.shape)

(800, 48, 48, 3) (800, 192, 192, 3) (200, 48, 48, 3) (200, 192, 192, 3)


### Load test data

In [16]:
def load_test_data(directory, ext, number_of_images = 100):
    files = load_data_from_dirs(load_path(directory), ext)
    
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()
        
    x_test_lr = lr_images(files, 4)
    x_test_lr = normalize(x_test_lr)
    
    return x_test_lr

In [17]:
def load_test_data_for_model(directory, ext, number_of_images = 100):

    files = load_data_from_dirs(load_path(directory), ext)
    
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()
        
    x_test_hr = hr_images(files)
    x_test_hr = normalize(x_test_hr)
    
    x_test_lr = lr_images(files, 4)
    x_test_lr = normalize(x_test_lr)
    
    return x_test_lr, x_test_hr


# Plot functions

In [18]:
#while training save generated image
#saves one image at a time
def plot_generated_images(output_dir, epoch, generator, x_test_hr, x_test_lr , dim=(1, 3), figsize=(15, 5)):
    
    examples = x_test_hr.shape[0]
    print(examples)
    value = randint(0, examples)
    image_batch_hr = denormalize(x_test_hr)
    image_batch_lr = x_test_lr
    gen_img = generator.predict(image_batch_lr)
    generated_image = denormalize(gen_img)
    image_batch_lr = denormalize(image_batch_lr)
    
    plt.figure(figsize=figsize)
    
    plt.subplot(dim[0], dim[1], 1)
    plt.imshow(image_batch_lr[value], interpolation='nearest')
    plt.axis('off')
        
    plt.subplot(dim[0], dim[1], 2)
    plt.imshow(generated_image[value], interpolation='nearest')
    plt.axis('off')
    
    plt.subplot(dim[0], dim[1], 3)
    plt.imshow(image_batch_hr[value], interpolation='nearest')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir + 'generated_image_%d.png' % epoch)
    
    

In [19]:
# Plots and save generated images(in form LR, SR, HR) from model to test the model 
# Save output for all images given for testing  
def plot_test_generated_images_for_model(output_dir, generator, x_test_hr, x_test_lr , dim=(1, 3), figsize=(15, 5)):
    
    examples = x_test_hr.shape[0]
    image_batch_hr = denormalize(x_test_hr)
    image_batch_lr = x_test_lr
    gen_img = generator.predict(image_batch_lr)
    generated_image = denormalize(gen_img)
    image_batch_lr = denormalize(image_batch_lr)
    
    for index in range(examples):
    
        plt.figure(figsize=figsize)
    
        plt.subplot(dim[0], dim[1], 1)
        plt.imshow(image_batch_lr[index], interpolation='nearest')
        plt.axis('off')
        
        plt.subplot(dim[0], dim[1], 2)
        plt.imshow(generated_image[index], interpolation='nearest')
        plt.axis('off')
    
        plt.subplot(dim[0], dim[1], 3)
        plt.imshow(image_batch_hr[index], interpolation='nearest')
        plt.axis('off')
    
        plt.tight_layout()
        plt.savefig(output_dir + 'test_generated_image_%d.png' % index)

In [20]:
# Takes LR images and save respective HR images
def plot_test_generated_images(output_dir, generator, x_test_lr, figsize=(5, 5)):
    
    examples = x_test_lr.shape[0]
    image_batch_lr = denormalize(x_test_lr)
    gen_img = generator.predict(image_batch_lr)
    generated_image = denormalize(gen_img)
    
    for index in range(examples):
    
        #plt.figure(figsize=figsize)
    
        plt.imshow(generated_image[index], interpolation='nearest')
        plt.axis('off')
        
        plt.tight_layout()
        plt.savefig(output_dir + 'high_res_result_image_%d.png' % index)
    
        plt.show()

# Model architecture

In [21]:
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import add
from keras.layers import Dense
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.applications.vgg19 import VGG19
import keras.backend as K
from keras.optimizers import Adam


In [22]:
# Subpixel Conv will upsample from (h, w, c) to (h/r, w/r, c/r^2)
def SubpixelConv2D(input_shape, scale=4):
    def subpixel_shape(input_shape):
        dims = [input_shape[0],input_shape[1] * scale,input_shape[2] * scale,int(input_shape[3] / (scale ** 2))]
        output_shape = tuple(dims)
        return output_shape
    
    def subpixel(x):
        return tf.depth_to_space(x, scale)
        
    return Lambda(subpixel, output_shape=subpixel_shape)

In [23]:
# Residual block
def res_block_gen(model,kernel_size,filters,strides):
    gen= model

    model= Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding='same')(model)
    model= BatchNormalization(momentum=0.5)(model)
    model= PReLU(alpha_initializer='zeros',alpha_regularizer=None,alpha_constraint=None,shared_axes=[1,2])(model)
    model= Conv2D(filters=filters, kernel_size=kernel_size,strides=strides,padding='same')(model)
    model= BatchNormalization(momentum=0.5)(model)

    model=add([gen,model])

    return model

In [24]:
# Upsampling block
def up_sampling_block(model,kernel_size,filters,strides):
    model= Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding='same')(model)
    model= UpSampling2D(size=2)(model)
    model= LeakyReLU(alpha=0.2)(model)
    return model

In [25]:
# Generator model
class Generator(object):
    def __init__(self,noise_shape):
        self.noise_shape=noise_shape

    def generator(self):
        gen_input= Input(shape=self.noise_shape)
        
        model=Conv2D(filters=64,kernel_size=9,strides=1,padding='same')(gen_input)
        model=PReLU(alpha_initializer='zeros',alpha_regularizer=None,alpha_constraint=None, shared_axes=[1,2])(model)

        gen_model= model

        for _ in range(16):
            model= res_block_gen(model,3,64,1)

        model=Conv2D(filters=64,kernel_size=3,strides=1,padding='same')(model)
        model=BatchNormalization(momentum=0.5)(model)
        model=add([gen_model,model])

        for _ in range(2):
            model=up_sampling_block(model,3,256,1)

        model= Conv2D(filters=3,kernel_size=9,strides=1,padding='same')(model)
        model=Activation('tanh')(model)

        generator_model= Model(inputs=gen_input,outputs=model)
        return generator_model

In [26]:
# Discriminator block
def discriminator_block(model,filters,kernel_size,strides):
    model=Conv2D(filters=filters, kernel_size=kernel_size,strides=strides,padding='same')(model)
    model= BatchNormalization(momentum=0.5)(model)
    model=LeakyReLU(alpha=0.2)(model)
    return model

In [27]:
# Discrimintaor model
class Discriminator(object):
    def __init__(self,image_shape):
        self.image_shape=image_shape

    def discriminator(self):
        
        dis_input = Input(shape=self.image_shape)

        model = Conv2D(filters=64, kernel_size=3, strides=1,padding='same')(dis_input)
        model = LeakyReLU(alpha=0.2)(model)

        model = discriminator_block(model,64,3,2)
        model = discriminator_block(model,128,3,1)
        model = discriminator_block(model,128,3,2)
        model = discriminator_block(model,256,3,1)
        model = discriminator_block(model,256,3,2)
        model = discriminator_block(model,512,3,1) 
        model = discriminator_block(model,512,3,2)

        model = Flatten()(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha = 0.2)(model)

        model = Dense(1)(model)
        model = Activation('sigmoid')(model)

        discriminator_model = Model(inputs = dis_input, outputs = model)

        return discriminator_model        

# Loss functions

In [28]:
class VGG_LOSS(object):

    def __init__(self, image_shape):
        
        self.image_shape = image_shape

    # computes VGG loss or content loss
    def vgg_loss(self, y_true, y_pred):
    
        vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=self.image_shape)
        vgg19.trainable = False
        # Make trainable as False
        for l in vgg19.layers:
            l.trainable = False
        model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
        model.trainable = False
    
        return K.mean(K.square(model(y_true) - model(y_pred)))

In [29]:
# together
def get_gan_network(discriminator,shape,generator,optimizer,vgg_loss):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=[x,gan_output])
    gan.compile(loss=[vgg_loss,'binary_crossentropy'],loss_weights=[1.,1e-3],optimizer=optimizer)
    return gan

In [30]:
def get_optimizer():
    adam= Adam(lr=1E-4, beta_1=0.9,beta_2=0.999, epsilon=1e-08)
    return adam

# Training



In [31]:
from tqdm import tqdm
import numpy as np

In [32]:
"""
downscale_factor=4
image_shape= (384,384,3)
#train function
def train(epochs,batch_size,input_dir,model_save_dir,number_of_images,train_test_ratio):

    # load training and testing data
    x_train_lr, x_train_hr, x_test_lr, x_test_hr = load_training_data(input_dir, '.jpg', number_of_images, train_test_ratio)

    batch_count = int(x_train_hr.shape[0]/batch_size)
    shape= (image_shape[0]//downscale_factor,image_shape[1]//downscale_factor,image_shape[2])

    generator = Generator(shape).generator()
    dsicriminator = Discrimintaor(image_shape).discriminator()

    optimizer = get_optimizer()
    generator.compile(loss=vgg_loss,optimizer=optimizer)
    discriminator.compile(loss='binary_crossentropy',optimizer=optimizer)

    gan = get_gan_network(discriminator,shape,generator,optimizer,vgg_loss)

    loss_file = open(model_save_dir + 'losses.txt', 'w+')
    loss_file.close()

    for e in range(1,epochs+1):
        print('-'*15,'Epoch %d' % e, '-'*15)

        for _ in tqdm(range(batch_count)):

            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]
            generated_images_sr = generator.predict(image_batch_lr)

            real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            fake_data_Y = np.random.random_sample(batch_size)*0.2
            
            discriminator.trainable = True
            
            d_loss_real = discriminator.train_on_batch(image_batch_hr, real_data_Y)
            d_loss_fake = discriminator.train_on_batch(generated_images_sr, fake_data_Y)
            discriminator_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]

            gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            discriminator.trainable = False
            gan_loss = gan.train_on_batch(image_batch_lr, [image_batch_hr,gan_Y])

        print("discriminator_loss : %f" % discriminator_loss)
        print("gan_loss :", gan_loss)
        gan_loss = str(gan_loss)
        
        loss_file = open(model_save_dir + 'losses.txt' , 'a')
        loss_file.write('epoch%d : gan_loss = %s ; discriminator_loss = %f\n' %(e, gan_loss, discriminator_loss) )
        loss_file.close()

        if e == 1 or e % 5 == 0:
            plot_generated_images(output_dir, e, generator, x_test_hr, x_test_lr)
        if e % 500 == 0:
            generator.save(model_save_dir + 'gen_model%d.h5' % e)
            discriminator.save(model_save_dir + 'dis_model%d.h5' % e)

"""

'\ndownscale_factor=4\nimage_shape= (384,384,3)\n#train function\ndef train(epochs,batch_size,input_dir,model_save_dir,number_of_images,train_test_ratio):\n\n    # load training and testing data\n    x_train_lr, x_train_hr, x_test_lr, x_test_hr = load_training_data(input_dir, \'.jpg\', number_of_images, train_test_ratio)\n\n    batch_count = int(x_train_hr.shape[0]/batch_size)\n    shape= (image_shape[0]//downscale_factor,image_shape[1]//downscale_factor,image_shape[2])\n\n    generator = Generator(shape).generator()\n    dsicriminator = Discrimintaor(image_shape).discriminator()\n\n    optimizer = get_optimizer()\n    generator.compile(loss=vgg_loss,optimizer=optimizer)\n    discriminator.compile(loss=\'binary_crossentropy\',optimizer=optimizer)\n\n    gan = get_gan_network(discriminator,shape,generator,optimizer,vgg_loss)\n\n    loss_file = open(model_save_dir + \'losses.txt\', \'w+\')\n    loss_file.close()\n\n    for e in range(1,epochs+1):\n        print(\'-\'*15,\'Epoch %d\' % e,

In [33]:

#train function
def train(x_train_lr, x_train_hr, x_test_lr, x_test_hr,epochs,batch_size,model_save_dir):
    downscale_factor=4
    image_shape= (192,192,3)

    loss = VGG_LOSS(image_shape)

    batch_count = int(x_train_hr.shape[0]/batch_size)
    shape= (image_shape[0]//downscale_factor,image_shape[1]//downscale_factor,image_shape[2])

    generator = Generator(shape).generator()
    discriminator = Discriminator(image_shape).discriminator()

    optimizer = get_optimizer()
    generator.compile(loss=loss.vgg_loss,optimizer=optimizer)
    discriminator.compile(loss='binary_crossentropy',optimizer=optimizer)

    gan = get_gan_network(discriminator,shape,generator,optimizer,loss.vgg_loss)

    loss_file = open(model_save_dir + 'losses.txt', 'w+')
    loss_file.close()

    for e in range(1,epochs+1):
        print('-'*15,'Epoch %d' % e, '-'*15)

        for _ in tqdm(range(batch_count)):

            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]
            generated_images_sr = generator.predict(image_batch_lr)

            real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            fake_data_Y = np.random.random_sample(batch_size)*0.2
            
            discriminator.trainable = True
            
            d_loss_real = discriminator.train_on_batch(image_batch_hr, real_data_Y)
            d_loss_fake = discriminator.train_on_batch(generated_images_sr, fake_data_Y)
            discriminator_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]

            gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            discriminator.trainable = False
            gan_loss = gan.train_on_batch(image_batch_lr, [image_batch_hr,gan_Y])

        print("discriminator_loss : %f" % discriminator_loss)
        print("gan_loss :", gan_loss)
        gan_loss = str(gan_loss)
        
        loss_file = open(model_save_dir + 'losses.txt' , 'a')
        loss_file.write('epoch%d : gan_loss = %s ; discriminator_loss = %f\n' %(e, gan_loss, discriminator_loss) )
        loss_file.close()

        if e == 1 or e % 5 == 0:
            plot_generated_images(output_dir, e, generator, x_test_hr, x_test_lr)
        if e % 500 == 0:
            generator.save(model_save_dir + 'gen_model%d.h5' % e)
            discriminator.save(model_save_dir + 'dis_model%d.h5' % e)



In [ ]:
epochs= 3000
batch_size= 16
model_save_dir='/content/drive/My Drive/intern notebooks/'
train(x_train_lr, x_train_hr, x_test_lr, x_test_hr,epochs,batch_size,model_save_dir)